# Spiking neural networks for EMG data streaming

In [ ]:
import os
import sys
sys.path.append("..")
sys.path.insert(0,"/s2net-emg")

!pip install librosa==0.9.1
!pip install pandas
!pip install numpy==1.21.4
!pip install numba==0.53.0
!pip install scipy
!pip install torch
!pip install torchvision
!pip install matplotlib
import scipy
from scipy import signal
import time
import numpy as np
import librosa
import torch
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from torch.utils.data import DataLoader
import torchvision


from utils import plot_spk_rec, plot_mem_rec
from models import SNN, SpikingConv2DLayer, ReadoutLayer, SurrogateHeaviside, SpikingDenseLayer

from data import EMGDataset, PSD, STFT, PSDNoDelta, NoFeatureExtraction, Rescale
from optim import RAdam

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# Different feature extraction functions
# 0 - No feature extraction
# 1 - PSD
# 2 - STFT
# 3 - PSDNoDelta
MODE = 1
# Load pretrained model?
LOAD = False

In [ ]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Loading Data

In [ ]:
train_data_root = "../data/train"
test_data_root = "../data/test"

training_words = os.listdir(train_data_root)
training_words = [x for x in training_words if os.path.isdir(os.path.join(train_data_root,x))]
training_words = [x for x in training_words if os.path.isdir(os.path.join(train_data_root,x)) if x[0] != "_" ]
print("{} training words:".format(len(training_words)))
print(training_words)

In [ ]:
testing_words = os.listdir(test_data_root)
testing_words = [x for x in testing_words if os.path.isdir(os.path.join(train_data_root,x))]
testing_words = [x for x in testing_words if os.path.isdir(os.path.join(train_data_root,x)) 
                 if x[0] != "_"]
print("{} testing words:".format(len(testing_words)))
if len(testing_words) != len(training_words):
    raise ValueError("Mismathed data")
print(testing_words)

In [ ]:
# Using labels from datasets
lb = sorted(testing_words)
label_dct = {lb[i]: i for i in range(len(lb))}
if len(testing_words) != len(label_dct.keys()):
    raise ValueError("Mismathed data")
print("label_dct:")
print(label_dct)

In [ ]:
sr = 150
size = 150
n_fft = size
hop_length = 20

In [ ]:
stft = STFT(n_fft, hop_length)
psd = PSD(sr, n_fft)
psdnodelta = PSDNoDelta(sr, n_fft)
nfe = NoFeatureExtraction()

rescale = Rescale()

if MODE == 0:
    transform = torchvision.transforms.Compose([nfe, rescale])
    in_chan = 1
    in_sh = 8
elif MODE == 1:
    transform = torchvision.transforms.Compose([psd, rescale])
    in_chan = 2
    in_sh = 8
elif MODE == 2:
    transform = torchvision.transforms.Compose([stft, rescale])
    in_chan = 8
    in_sh = 52
elif MODE == 3:
    transform = torchvision.transforms.Compose([psdnodelta, rescale])
    in_chan = 1
    in_sh = 8
else:
    print("Incorect mode, using no transform")
    transform = torchvision.transforms.Compose([nfs, rescale])
    in_chan = 1
    in_sh = 8



def collate_fn(data):
    
    X_batch = np.array([d[0] for d in data])
    std = X_batch.std(axis=(0,3), keepdims=True)
    X_batch = torch.tensor(X_batch/std)
    
    y_batch = torch.tensor([d[1] for d in data])
    
    return X_batch, y_batch 
 

In [ ]:
batch_size = 20

train_dataset = EMGDataset(train_data_root, label_dct, transform = transform, mode="train", max_nb_per_class=None)
train_sampler = torch.utils.data.WeightedRandomSampler(train_dataset.weights,len(train_dataset.weights))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, sampler=train_sampler, collate_fn=collate_fn)

test_dataset = EMGDataset(test_data_root, label_dct, transform = transform, mode="test")
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)

# Creating the model

In [ ]:
spike_fn = SurrogateHeaviside.apply

w_init_std = 0.15
w_init_mean = 0.

w_init_std = 0.15
w_init_mean = 0.


layers = []
in_channels = in_chan
out_channels = 64
kernel_size = (4,3)
dilation = (1,1)
input_shape = in_sh
output_shape = input_shape # padding mode is "same"
layers.append(SpikingConv2DLayer(input_shape, output_shape,
                 in_channels, out_channels, kernel_size, dilation,
                 spike_fn, w_init_mean=w_init_mean, w_init_std=w_init_std, recurrent=False,
                               lateral_connections=True))

in_channels = out_channels
out_channels = 64
kernel_size = (4,3)
dilation = (4,3)
input_shape = output_shape
output_shape = input_shape # padding mode is "same"
layers.append(SpikingConv2DLayer(input_shape, output_shape,
                 in_channels, out_channels, kernel_size, dilation,
                 spike_fn, w_init_mean=w_init_mean, w_init_std=w_init_std, recurrent=False,
                              lateral_connections=True))

in_channels = out_channels
out_channels = 64
kernel_size = (4,3)
dilation = (4,3)
input_shape = output_shape
output_shape = input_shape # padding mode is "same"
layers.append(SpikingConv2DLayer(input_shape, output_shape,
                 in_channels, out_channels, kernel_size, dilation,
                 spike_fn, w_init_mean=w_init_mean, w_init_std=w_init_std, recurrent=False,
                               lateral_connections=True, flatten_output=True))

# previous layer output has been flattened
input_shape = output_shape*out_channels
output_shape = 10
time_reduction="mean" #mean or max
layers.append(ReadoutLayer(input_shape, output_shape,
                 w_init_mean=w_init_mean, w_init_std=w_init_std, time_reduction=time_reduction))

snn = SNN(layers).to(device, dtype)

X_batch, _ = next(iter(train_dataloader))
X_batch = X_batch.to(device, dtype)
snn(X_batch)

for i,l in enumerate(snn.layers):
    if isinstance(l, SpikingDenseLayer) or isinstance(l, SpikingConv2DLayer):
        print("Layer {}: average number of spikes={:.4f}".format(i,l.spk_rec_hist.mean()))

In [ ]:
#LOADING MODEL
if LOAD:
    if MODE == 1:
        snn.load_state_dict(torch.load("../saved/modeldictPSD.pt", map_location=device))
    elif MODE == 2:
        snn.load_state_dict(torch.load("../saved/modeldictSTFT.pt", map_location=device))
    elif MODE == 3:
        snn.load_state_dict(torch.load("../saved/modeldictPSDNoDelta.pt", map_location=device))  
    else:
        snn.load_state_dict(torch.load("../saved/modeldictNOFE.pt", map_location=device))


# Training and plotting

In [ ]:
nb_plt = 9

batch_idx = np.random.choice(batch_size, nb_plt, replace=False)

# Plotting spike trains or membrane potential
for i,l in enumerate(snn.layers):
    
    if isinstance(l, SpikingDenseLayer):
        spk_rec = l.spk_rec_hist
        plot_spk_rec(spk_rec, idx=batch_idx)
    elif isinstance(l, SpikingConv2DLayer):
        spk_rec = l.spk_rec_hist
        plot_spk_rec(spk_rec.sum(1), idx=batch_idx)
    else:
        mem_rec = l.mem_rec_hist
        plot_mem_rec(mem_rec, batch_idx)

In [ ]:
def train(model, params, optimizer, train_dataloader, reg_loss_coef, nb_epochs, scheduler=None, warmup_epochs=0):
    
    log_softmax_fn = torch.nn.LogSoftmax(dim=1)
    loss_fn = torch.nn.NLLLoss()
    
    if warmup_epochs > 0:
        for g in optimizer.param_groups:
            g['lr'] /= len(train_dataloader)*warmup_epochs
        warmup_itr = 1
    
    hist = {'loss':[]}
    for e in range(nb_epochs):
        local_loss = []
        reg_loss = [[] for _ in range(len(model.layers)-1)]
        
        for x_batch, y_batch in train_dataloader:

            x_batch = x_batch.to(device, dtype)
            y_batch = y_batch.to(device)

            output, loss_seq = model(x_batch)
            log_p_y = log_softmax_fn(output)
            loss_val = loss_fn(log_p_y, y_batch)
            local_loss.append(loss_val.item())

            for i,loss in enumerate(loss_seq[:-1]):
                reg_loss_val = reg_loss_coef*loss*(i+1)/len(loss_seq[:-1])
                loss_val += reg_loss_val
                reg_loss[i].append(reg_loss_val.item())


            optimizer.zero_grad()
            loss_val.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), 5)
            optimizer.step()
            model.clamp()

            if e < warmup_epochs:
                for g in optimizer.param_groups:
                    g['lr'] *= (warmup_itr+1)/(warmup_itr)
                warmup_itr += 1
                
                
        if scheduler is not None and e >= warmup_epochs:
            scheduler.step()
        
        mean_loss = np.mean(local_loss)
        hist['loss'].append(mean_loss)
        print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        
        for i,loss in enumerate(reg_loss):
            mean_reg_loss = np.mean(loss)
            print("Layer %i: reg loss=%.5f"%(i,mean_reg_loss))
            
        for i,l in enumerate(snn.layers[:-1]):
            print("Layer {}: average number of spikes={:.4f}".format(i,l.spk_rec_hist.mean()))
        

    return hist
        
def compute_classification_accuracy(model, dataloader):
    accs = []
    with torch.no_grad():
        for x_batch, y_batch in dataloader:

            x_batch = x_batch.to(device, dtype)
            y_batch = y_batch.to(device)
            output, _ = model(x_batch)
            _,am=torch.max(output,1) # argmax over output units
            tmp = np.mean((y_batch==am).detach().cpu().numpy()) # compare to labels
            accs.append(tmp)
    return np.mean(accs)

In [ ]:
if LOAD == False:
    lr = 1e-3
    weight_decay = 1e-5
    reg_loss_coef = 0.1
    nb_epochs = 10
    warmup_epochs = 1

    params = [{'params':l.w, 'lr':lr, "weight_decay":weight_decay } for i,l in enumerate(snn.layers)]
    params += [{'params':l.v, 'lr':lr, "weight_decay":weight_decay} for i,l in enumerate(snn.layers[:-1]) if l.recurrent]
    params += [{'params':l.b, 'lr':lr} for i,l in enumerate(snn.layers)]
    if snn.layers[-1].time_reduction == "mean":
        params += [{'params':l.beta, 'lr':lr} for i,l in enumerate(snn.layers[:-1])]
    elif snn.layers[-1].time_reduction == "max":
        params += [{'params':l.beta, 'lr':lr} for i,l in enumerate(snn.layers)]
    else:
        raise ValueError("Readout time recution should be 'max' or 'mean'")
    
    optimizer = RAdam(params)
 
    gamma = 0.85
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma, last_epoch=-1)

    hist = train(snn, params, optimizer, train_dataloader, reg_loss_coef, nb_epochs=nb_epochs,
                    scheduler=scheduler, warmup_epochs=1)


In [ ]:
test_accuracy = compute_classification_accuracy(snn, test_dataloader)
print("Test accuracy=%.3f"%(test_accuracy))

In [ ]:
snn(X_batch)

# Plotting spike trains or membrane potential
for i,l in enumerate(snn.layers):
    
    if isinstance(l, SpikingDenseLayer):
        print("Layer {}: average number of spikes={:.4f}".format(i,l.spk_rec_hist.mean()))
        spk_rec = l.spk_rec_hist
        plot_spk_rec(spk_rec, idx=batch_idx)
    elif isinstance(l, SpikingConv2DLayer):
        print("Layer {}: average number of spikes={:.4f}".format(i,l.spk_rec_hist.mean()))
        spk_rec = l.spk_rec_hist
        plot_spk_rec(spk_rec.sum(1), idx=batch_idx)
    else:
        mem_rec = l.mem_rec_hist
        plot_mem_rec(mem_rec, batch_idx)

# Streaming data test

In [ ]:
from streamload import EMGStream, PSD, STFT, PSDNoDelta, NoFeatureExtraction, Rescale

def compute_classification_accuracy(model, dataloader):
    accs = []
    cnt = 0
    
    with torch.no_grad():
        for x_batch, y_batch in dataloader:

            x_batch = x_batch.to(device, dtype)
            y_batch = y_batch.to(device)
            output, _ = model(x_batch)
            _,am=torch.max(output,1) # argmax over output units
            tmp = np.mean((y_batch==am).detach().cpu().numpy()) # compare to labels
            accs.append(tmp)

            if cnt%15 == 0:
                print("Estimation: ", tmp, "SNN output: ", am.item(), "Label: ", y_batch.item())
                print("Batches estimated: ", cnt)
                print("Avg accuracy: %0.3f"%np.mean(accs))
            cnt+=1

    return np.mean(accs)

fn = "../raw/2_raw_data_1.txt"
bat_size = 15
buff_size = 150  # a multiple of bat_size

stft = STFT(n_fft, hop_length)
psd = PSD(sr, n_fft)
psdnodelta = PSDNoDelta(sr, n_fft)
nfe = NoFeatureExtraction()

rescale = Rescale()

if MODE == 0:
    transform = torchvision.transforms.Compose([nfe, rescale])
    in_chan = 1
    in_sh = 8
elif MODE == 1:
    transform = torchvision.transforms.Compose([psd, rescale])
    in_chan = 2
    in_sh = 8
elif MODE == 2:
    transform = torchvision.transforms.Compose([stft, rescale])
    in_chan = 8
    in_sh = 52
elif MODE == 3:
    transform = torchvision.transforms.Compose([psdnodelta, rescale])
    in_chan = 1
    in_sh = 8
else:
    print("Incorect mode, using no transform")
    transform = torchvision.transforms.Compose([nfs, rescale])
    in_chan = 1
    in_sh = 8



stream_loader = EMGStream(fn, bat_size, buff_size, transform=transform, shuffle=False)
test_accuracy = compute_classification_accuracy(snn, stream_loader)
print("Test accuracy=%.3f"%(test_accuracy))
